In [1]:
from konlpy.tag import Kkma
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
from configs import DEFINES

ModuleNotFoundError: No module named 'configs'

In [2]:
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]

In [3]:
# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리
def load_data():
    db = pd.read_csv("./data_in/ChatBotData.csv")
    train, test = train_test_split(db)
    train_q, train_a = train["Q"], train["A"]
    test_q, test_a = test["Q"], test["A"]
    
    return train_q, train_a, test_q, test_a

In [4]:
# Req 1-1-2. 텍스트 데이터에 정규화를 사용하여 ([~.,!?\"':;)(]) 제거
def prepro_noise_canceling(data):
    data = re.sub("[~.,!?\"':;)(]", "", data)

    return data

In [5]:
# Req 1-1-3. 텍스트 데이터에 토크나이징
def tokenizing_data(data):
    kkma = Kkma()
    tokens_list = []
    
    for d in tqdm(data):
        tokens = " ".join(kkma.morphs(d.replace(" ", "")))
        tokens_list.append(tokens)
    
    return tokens_list

In [1]:
# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리
def enc_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)
    
    for seq in value:
        
        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]
        
        for word in seq.split():
            if dictionary.get(word) is not None:
                # seq_index에 dictionary 안의 인덱스를 extend 한다
                seq_index.extend(dictionary[word])
            else:
                # dictionary에 존재 하지 않는 다면 UNK 값을 extend 한다 
                seq_index.extend(dictionary[UNK])
                
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]
            
        # seq의 길이를 저장
        seq_len.append(len(seq_index))
        
        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += (DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]
        
        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)
        
    return np.asarray(seq_input_index), seq_len